## Google Maps Geocoding API
Creating API keys
The API key is a unique identifier that authenticates requests associated with your project for usage and billing purposes. You must have at least one API key associated with your project.

To create an API key:
https://developers.google.com/maps/documentation/geocoding/get-api-key#:~:text=Go%20to%20the%20Google%20Maps%20Platform%20%3E%20Credentials%20page.&text=On%20the%20Credentials%20page%2C%20click,Click%20Close.

Install

In [ ]:
!pip install -U googlemaps

# import

In [3]:
import pandas as pd
import requests
from tqdm import tqdm

# Initialize the Google Maps Geocoding API endpoint and API key

In [11]:


API_KEY = 'Add-Your-API-key'
API_ENDPOINT = 'https://maps.googleapis.com/maps/api/geocode/json'


# Import the CSV file

In [12]:
# Import the CSV file
df = pd.read_csv('./data/output/sample_data_Fuzzy_result.csv')

In [13]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,Acc,ID,Center Names,Submitted_by,Biosample_Submission,Institution,Country/territory,Score
0,0,0.0,SRR8606903,7314038.0,AUSTRALIAN INSTITUTE OF MARINCE SCIENCES,Australian Institute of Marince sciences,"Australian Institute of Marince sciences, Patr...",Australian Institute of Marine Science (AIMS),Australia,92.0
1,0,1.0,SRR11788653,10848896.0,CLARK UNIVERSITY,Clark University,"Clark University, Nathan Ahlgren",Clark University,United States of America (USA),100.0
2,0,2.0,SRR9158324,7959983.0,CALIFORNIA STATE UNIVERSITY FULLERTON,California State University Fullerton,"California State University Fullerton, Maria R...","California State University, Fullerton (CSUF)",United States of America (USA),94.0
3,0,6.0,SRR10018586,8900739.0,CFSAN,FDA Center for Food Safety and Applied Nutrit...,CFSAN,NaN,NaN,NaN
4,0,8.0,SRR10406092,9345478.0,AUSTRALIAN INSTITUTE OF MARINE SCIENCE,Australian Institute of Marine Science,"Australian Institute of Marine Science, Heidi ...",Australian Institute of Marine Science (AIMS),Australia,94.0


# Create new columns for latitude, longitude, and full address

In [14]:
# Create new columns for latitude, longitude, and full address
df['Latitude'] = ''
df['Longitude'] = ''
df['Full Address'] = ''
df.head(1)

,Unnamed: 0.1,Unnamed: 0,Acc,ID,Center Names,Submitted_by,Biosample_Submission,Institution,Country/territory,Score,Latitude,Longitude,Full Address
0,0,0.0,SRR8606903,7314038.0,AUSTRALIAN INSTITUTE OF MARINCE SCIENCES,Australian Institute of Marince sciences,"Australian Institute of Marince sciences, Patr...",Australian Institute of Marine Science (AIMS),Australia,92.0,,,


In [17]:
# Geocode each address using the Google Maps Geocoding API
for i in tqdm(range(len(df))):
    try:
        full_address = f"{df['Institution'][i]}, {df['Country/territory'][i]}"
        params = {'address': full_address, 'key': API_KEY}
        response = requests.get(API_ENDPOINT, params=params)
        json_response = response.json()
        if json_response['status'] == 'OK':
            df.loc[i, 'Latitude'] = json_response['results'][0]['geometry']['location']['lat']
            df.loc[i, 'Longitude'] = json_response['results'][0]['geometry']['location']['lng']
            df.loc[i, 'Full Address'] = json_response['results'][0]['formatted_address']
    except:
        # If geocoding fails, set the latitude, longitude, and full address columns to empty strings
        df.loc[i, 'Latitude'] = ''
        df.loc[i, 'Longitude'] = ''
        df.loc[i, 'Full Address'] = ''

100%|██████████| 32/32 [00:09<00:00,  3.46it/s]


In [20]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,Acc,ID,Center Names,Submitted_by,Biosample_Submission,Institution,Country/territory,Score,Latitude,Longitude,Full Address
0,0,0.0,SRR8606903,7314038.0,AUSTRALIAN INSTITUTE OF MARINCE SCIENCES,Australian Institute of Marince sciences,"Australian Institute of Marince sciences, Patr...",Australian Institute of Marine Science (AIMS),Australia,92.0,-25.274398,133.775136,Australia
1,0,1.0,SRR11788653,10848896.0,CLARK UNIVERSITY,Clark University,"Clark University, Nathan Ahlgren",Clark University,United States of America (USA),100.0,42.252035,-71.824538,"950 Main St, Worcester, MA 01610, USA"
2,0,2.0,SRR9158324,7959983.0,CALIFORNIA STATE UNIVERSITY FULLERTON,California State University Fullerton,"California State University Fullerton, Maria R...","California State University, Fullerton (CSUF)",United States of America (USA),94.0,33.882348,-117.885103,"800 N State College Blvd, Fullerton, CA 92831,..."
3,0,6.0,SRR10018586,8900739.0,CFSAN,FDA Center for Food Safety and Applied Nutrit...,CFSAN,NaN,NaN,NaN,,,
4,0,8.0,SRR10406092,9345478.0,AUSTRALIAN INSTITUTE OF MARINE SCIENCE,Australian Institute of Marine Science,"Australian Institute of Marine Science, Heidi ...",Australian Institute of Marine Science (AIMS),Australia,94.0,-25.274398,133.775136,Australia


# Save the final result

In [21]:
df.to_csv('./data/output/sample_data_Adress.csv', index=False)